In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import linear_model
import warnings

import time
from sklearn.feature_extraction import DictVectorizer
from scipy import sparse
import pickle

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import pandas as pd
from pandas import Series

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
from datetime import datetime
%matplotlib inline
t = time.time()
#elapsed = (time.time() - t)/60

In [4]:
X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
X4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_train2.dat")
X5=pd.read_pickle("./mostrecentinput/cat_le_train2.dat") 
X=np.hstack((X,X1,X2,X3,X4,X5))

y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))

In [ ]:
class RandomForestClassifierWithCoef(RandomForestClassifier):
    def fit(self, *args, **kwargs):
        super(RandomForestClassifierWithCoef, self).fit(*args, **kwargs)
        self.coef_ = self.feature_importances_

#-----------------------------------------------
rf = RandomForestClassifierWithCoef(n_estimators=50, min_samples_leaf=5, n_jobs=-1)
rfecv = RFECV(estimator=rf, step=40, cv=2, scoring='roc_auc', verbose=2)
selector=rfecv.fit(X, y)

Xselect=X[:,selector.support_]
clf = RandomForestClassifier(n_estimators=50, min_samples_leaf=5, n_jobs=-1)
skf = cross_validation.StratifiedKFold(y, n_folds=2) # K-fold cv splitting
for train0, cv0 in skf:
    train, cv = train0, cv0
    break

MODEL = clf.fit(Xselect[train,:], y[train])
Xselect[train,:].shape
AUC1=scorer(MODEL, Xselect[cv,:], y[cv])
str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='SDGRF_'  +  str1 + '_AUC_' + str(AUC1) + '.p'
np.arange(X.shape[1])[selector.support_].dump(filename)
print(Xselect[train,:].shape)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

In [ ]:
from sklearn import ensemble
class GradientBoostingClassifierWithCoef(ensemble.GradientBoostingClassifier):
    def fit(self, *args, **kwargs):
        super(GradientBoostingClassifierWithCoef, self).fit(*args, **kwargs)
        self.coef_ = self.feature_importances_


original_params = {'n_estimators': 40, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}

params=original_params
GB = GradientBoostingClassifierWithCoef(**params)
rfecv = RFECV(estimator=GB, step=100, cv=2, scoring='roc_auc', verbose=2)
selector=rfecv.fit(X, y)

Xselect=X[:,selector.support_]
clf = ensemble.GradientBoostingClassifier(**params)
skf = cross_validation.StratifiedKFold(y, n_folds=2) # K-fold cv splitting

for train0, cv0 in skf:
    train, cv = train0, cv0
    break

MODEL = clf.fit(Xselect[train,:], y[train])
Xselect[train,:].shape
AUC1=scorer(MODEL, Xselect[cv,:], y[cv])
str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='SDGRF_'  +  str1 + '_AUC_' + str(AUC1) + '.p'
np.arange(X.shape[1])[selector.support_].dump(filename)
print(Xselect[train,:].shape)